<!-- <img  src="https://swan.web.cern.ch/sites/swan.web.cern.ch/files/pictures/logo_swan_letters.png" alt="SWAN" style="float: left; width: 15%; margin-right: 5%; margin-left: 17%; margin-top: 1.0em; margin-bottom: 2.0em;">
<img src="https://spark.apache.org/images/spark-logo-trademark.png" alt="EP-SFT" style="float: left; width: 25%; margin-right: 0%; margin-left: 0%; margin-bottom: 2.0em;">
<img src="https://cms-docdb.cern.ch/cgi-bin/PublicDocDB/RetrieveFile?docid=3045&filename=CMSlogo_color_label_1024_May2014.png&version=3" alt="CMS" style="float: left; width: 12%; margin-left: 5%; margin-right: 5%; margin-bottom: 2.0em;"> -->
<p style="clear: both;">
<div style="text-align:center"><h1>CMS H&#8594;µµ analysis  
     <br> with Coffea package from Fermilab</h1></div>
<div style="text-align:center"><i>Author: Dmitry Kondratyev, based on example code by Lindsey Gray<br> and adapted for the demonstration by Stefan Piperov</i></div>
<hr style="border-top-width: 4px; border-top-color: #34609b;">

# Search for Higgs boson decaying into two muons

This code uses awkward array toolset, and utilizing Coffea [histograms](https://coffeateam.github.io/coffea/modules/coffea.hist.html).
This also shows the analysis object syntax implemented by Coffea [JaggedCandidateArray](https://coffeateam.github.io/coffea/api/coffea.analysis_objects.JaggedCandidateMethods.html), and the usage of custom [accumulators](https://coffeateam.github.io/coffea/api/coffea.processor.AccumulatorABC.html) other than histograms.  Further, it introduces the [processor](https://coffeateam.github.io/coffea/api/coffea.processor.ProcessorABC.html) concept and the interface to apache spark.


Instructions to run on Hammer at Purdue:
===
It is assumed that this demonstration is being run in a Jupyter Lab session on Hammer, as described in the README file.  
  
**NOTE** You need a separate terminal from which to create and control your DASK cluster.

In [1]:
%%bash -l
hostname
pwd
ml anaconda/5.3.1-py37
source activate hmumu_coffea
voms-proxy-info

Prepare directories for output files  

**Note** these paths need to be changed also further down in "Out_dir" in the "Run the DASK Executor" cell

In [1]:
mkdir -p /depot/cms/hmm/coffea/Stefan/test_2018_test/unbinned
mkdir -p /depot/cms/hmm/coffea/Stefan/test_2018_test/binned

hammer-c026
/home/spiperov/dmitry/hmumu-coffea_01May2020
subject   : /DC=ch/DC=cern/OU=Organic Units/OU=Users/CN=piperov/CN=422973/CN=Stefcho Piperov/CN=951169794
issuer    : /DC=ch/DC=cern/OU=Organic Units/OU=Users/CN=piperov/CN=422973/CN=Stefcho Piperov
identity  : /DC=ch/DC=cern/OU=Organic Units/OU=Users/CN=piperov/CN=422973/CN=Stefcho Piperov
type      : RFC compliant proxy
strength  : 1024 bits
path      : /home/spiperov/x509up_u638764
timeleft  : 191:21:43


In [ ]:
import time
import coffea
print("Coffea version: ", coffea.__version__)
import socket

print(socket.gethostname())
from coffea import util
import coffea.processor as processor
import multiprocessing as mp
print(f"{mp.cpu_count()} CPUs")

Prepare data samples
===


In [ ]:
from python.samples_info import SamplesInfo
samples = [
### Data ###
    'data_A', #not available for 2016
    'data_B',
    'data_C',
    'data_D','data_E',
    'data_F',
    'data_G','data_H',

### Essential MC ###    
    'dy_m105_160_amc', 
    'dy_m105_160_vbf_amc',
#      'ggh_amcPS', 
#     'vbf_powhegPS', 
#     'ttjets_dl',
#    "ewk_lljj_mll105_160_ptj0",

### Non-essential MC ### 
#     'ttjets_sl',
#     'ttz',
#     'ttw',
#     'st_tw_top','st_tw_antitop',
#     'ww_2l2nu',
#     'wz_2l2q',
#     'wz_3lnu',
#     'wz_1l1nu2q',
#      'zz',
# # ##
    
]

purdue = 'root://xrootd.rcac.purdue.edu/'

year = '2018'
label = 'test' # change this to save to other directory

samp_info = SamplesInfo(year=year, out_path=f'test_{year}_{label}', server=purdue, debug=True)

# 'outer' refers to parallelization by sample, 'inner' - by ROOT file in each sample
samp_info.load(samples, nchunks=1, parallelize_outer=1, parallelize_inner=10)
samp_info.compute_lumi_weights()


Connect to Dask scheduler
===
**Note** that in the next cell, in the line  
`client = distributed.Client('128.211.149.132:36442')`  
you need to substitute the IP address and TCP port number of your previously started DASK scheduler (e.g. as reported by the "print(cluster)" command in the scheduler terminal)

In [ ]:
from dask.distributed import Client
import pytest
from coffea.processor.executor import dask_executor
import dask
from python.dimuon_processor import DimuonProcessor

try_cluster = True

if try_cluster:
    from dask_jobqueue import SLURMCluster
    distributed = pytest.importorskip("distributed", minversion="1.28.1")
    distributed.config['distributed']['worker']['memory']['target'] = 0.75
    distributed.config['distributed']['worker']['memory']['spill'] = 0.80
    distributed.config['distributed']['worker']['memory']['pause'] = 0.95
    distributed.config['distributed']['worker']['memory']['terminate'] = False
#    distributed.config['distributed']['worker']['memory']['terminate'] = 1.50
    distributed.config['distributed']['worker']['connections']['outgoing'] = 500
    distributed.config['distributed']['worker']['connections']['incoming'] = 100
    distributed.config['distributed']['scheduler']['allowed-failures'] = 5
    distributed.config['distributed']['scheduler']['bandwidth'] = 1000000000
    distributed.config['distributed']['scheduler']['work-stealing'] = False
    distributed.config['distributed']['scheduler']['default-task-durations'] = "1h"
    distributed.config['distributed']['comm']['retry']['count'] = 2
    distributed.config['distributed']['comm']['socket-backlog'] = 20480
    distributed.config['distributed']['comm']['offload'] = "100MiB"
    distributed.config['distributed']['deploy']['lost-worker-timeout'] = "1m"
    
    client = distributed.Client('128.211.149.132:36442')
    
else:
    n_workers = 46
    distributed = pytest.importorskip("distributed", minversion="1.28.1")
    distributed.config['distributed']['worker']['memory']['terminate'] = False
    client = distributed.Client(processes=True, dashboard_address=None, n_workers=n_workers, threads_per_worker=1, memory_limit='12GB') 

client

In [ ]:
client.scheduler_info()

In [ ]:
client.restart()


Run the Dask executor
===


In [ ]:
tstart = time.time()
print(client)

for ds_name, fileset_ in samp_info.filesets_chunked.items():
#    client.restart()
    for ichunk, ifileset in enumerate(fileset_):
        print(f"Processing {ds_name}, chunk {ichunk+1}/{samp_info.nchunks} ...")
        output = processor.run_uproot_job(ifileset, 'Events',\
                                      DimuonProcessor(samp_info=samp_info, do_jecunc=False),\
                                      dask_executor,\
                                      executor_args={'nano': True, 'client': client})

        #out_dir = f"/depot/cms/hmm/coffea/{samp_info.out_path}/"
        out_dir = f"/depot/cms/hmm/coffea/Stefan/{samp_info.out_path}/"
        #out_dir = f"/tmp/spiperov/hmm/coffea/{samp_info.out_path}/"

        try:
            os.mkdir(out_dir)
        except:
            pass

        for mode in output.keys():
            out_dir_ = f"{out_dir}/{mode}/"
            out_path_ = f"{out_dir_}/{ds_name}_{ichunk}.coffea"
            try:
                os.mkdir(out_dir_)
            except:
                pass
            util.save(output[mode], out_path_)

        output.clear()
        print(f"Saved output to {out_dir}")
    
elapsed = time.time() - tstart

print(f"Total time: {elapsed} s")

Plot Data/MC comparison
---

In [ ]:
import os,glob
import argparse
from python.postprocessing import postprocess, plot, save_shapes
from config.variables import variables
from config.datasets import datasets
import pandas as pd

year = '2018'
label = 'test'

to_plot = ['dimuon_mass']
# to_plot = ['dimuon_mass','mu1_pt']
vars_to_plot = {v.name:v for v in variables if v.name in to_plot}
samples = [
    'data_A',
    'data_B',
    'data_C',
    'data_D',
    'data_E',
    'data_F',
    'data_G',
    'data_H',
    'dy_m105_160_amc',
    'dy_m105_160_vbf_amc',
    'ewk_lljj_mll105_160_ptj0',
    'ttjets_dl',
    'ttjets_sl',
    'ttz',
    'ttw',
    'st_tw_top','st_tw_antitop',
    'ww_2l2nu',
    'wz_2l2q',
    'wz_3lnu',
    'zz',
    'ggh_amcPS',
    'vbf_powhegPS',
]


postproc_args = {
    'modules': ['to_pandas',  'get_hists'],
    'year': year,
    'label': label,
#    'in_path': f'/depot/cms/hmm/coffea/test_{year}_{label}/',
    'in_path': f'/depot/cms/hmm/coffea/Stefan/test_{year}_{label}/',
#    'in_path': f'/tmp/spiperov/hmm/coffea/test_{year}_{label}/',
    'syst_variations': ['nominal'],
    'samples':samples,
    'channels': ['vbf'],
    'regions': ['h-peak', 'h-sidebands'],
    'vars_to_plot': list(vars_to_plot.values()),
    'wgt_variations': False,
}


dfs, hist_dfs, edges = postprocess(postproc_args)
hist = {}
for var, hists in hist_dfs.items():
    hist[var] = pd.concat(hists, ignore_index=True)

plot(vars_to_plot['dimuon_mass'], hist, 'wgt_nominal', edges['dimuon_mass'], postproc_args, save=False, show=True, plotsize=8)
# for vname, var in vars_to_plot.items():
#     for r in postproc_args['regions']:
#        plot(var, hist, 'wgt_nominal', edges[vname], postproc_args, r, save=False, show=True, plotsize=8)